In [1]:
import os
import time
import numpy as np
import pickle
import torch
from torch.utils import data
from torch import nn, optim
from torch.nn import functional as F
from torchvision.models import resnet152
import matplotlib.pyplot as plt

import sys 
sys.path.append('..')
from dataset import NyuV2
from modeling import train_model

PyTorch Version:  0.4.1
Torchvision Version:  0.2.1


In [2]:
data_path = '../data/nyu_v2/'
batch_size = 16
# img_size = (224, 224)
depth_size = (25, 32)
# max_img_val = 255.0
# max_depth_val = 9.9955
n_epochs = 3
seed = 2
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
dataloaders = {
    'train': data.DataLoader(NyuV2(os.path.join(data_path, 'train')),
                               batch_size=batch_size, shuffle=True),
    'val': data.DataLoader(NyuV2(os.path.join(data_path, 'test')),
                              batch_size=batch_size, shuffle=True)
}

In [4]:
def init_ft_model(model):
    for param in model.parameters():
        param.requires_grad = False
    
    model.fc = nn.Linear(2048, depth_size[0] * depth_size[1])
#     params_to_update = model.parameters()
    
    print("Params to learn:")
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
            
    return model, params_to_update

In [5]:
model = resnet152(pretrained=True)
model, params_to_update = init_ft_model(model)
model = model.to(device)

Params to learn:
	 fc.weight
	 fc.bias


In [6]:
optimizer = optim.Adam(params_to_update, lr=1e-4)
criterion = nn.MSELoss(reduction='sum')

In [7]:
model, train_loss_history, val_loss_history = train_model(model, dataloaders,
                                                          criterion, optimizer,
                                                          n_epochs, device)

Epoch 0/2
----------


/anaconda/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Training:   Loss: 38.3439 RMSE: 6.1922
Validation: Loss: 33.0862 RMSE: 5.7521
Epoch complete in 0m 48s

Epoch 1/2
----------
Training:   Loss: 23.4274 RMSE: 4.8402
Validation: Loss: 25.5786 RMSE: 5.0575
Epoch complete in 0m 47s

Epoch 2/2
----------
Training:   Loss: 18.9105 RMSE: 4.3486
Validation: Loss: 22.5182 RMSE: 4.7453
Epoch complete in 0m 47s

Training complete in 2m 23s
Best val loss: 22.518163
